In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sb
import re
from itertools import combinations
from tqdm.notebook import tqdm
import glob
import datetime

plt.style.use("dark_background")

path = "C:/Windows/Fonts/malgun.ttf"
font_name = mpl.font_manager.FontProperties(fname=path).get_name()
mpl.rc("font", family=font_name)
mpl.rc("axes", unicode_minus=False)

# 협업플랫폼

## 기본적인 전처리

In [2]:
target_year = 2021
target_month = 5
pref = "D:/디지털혁신팀/태블로/협업플랫폼 사용량/협업플랫폼 사용량_FINAL/"

hr = pd.read_excel(f"{pref}HR/사원명단_{target_year}{str(target_month).zfill(2)}.xlsx")

hr = hr.drop_duplicates()
hr = hr.rename({"사번":"num", "성명":"name", "EMAIL":"email", "본부명":"hq", "조직코드":"code", "조직명":"dep", "겸직명1":"dep1", "겸직명2":"dep2", "겸직명3":"dep3", "겸직명4":"dep4", "겸직명5":"dep5", "직급":"rank", "구분":"type"}, axis=1)
hr = hr[["num", "name", "email", "hq", "dep", "dep1", "dep2", "dep3", "dep4", "dep5", "rank", "type"]]
hr[["name", "email", "hq", "dep", "dep1", "dep2", "dep3", "dep4", "dep5", "rank", "type"]] = hr[["name", "email", "hq", "dep", "dep1", "dep2", "dep3", "dep4", "dep5", "rank", "type"]].astype("str")

hr.loc[hr["name"]=="윤 해복", "hq"] = "개발영업본부"
hr.loc[hr["name"]=="서 효석", "hq"] = "개발영업본부"

# 휴직 상태인 직원은 제외합니다.
hr = hr[~hr["dep"].str.contains("휴직")]
# 현장의 "hq"는 "현장"으로 변환합니다.
hr.loc[hr["type"].str.contains("현장"), "hq"] = "현장"
# "그룹"은 "그룹장"으로 변경합니다.
# hr["dep"] = hr["dep"].apply(lambda x:x + "장" if "그룹" in x else x)
# 신규준비는 해당 팀 소속으로 간주합니다.
hr["dep"] = hr["dep"].str.replace("(신규준비)", "", regex=False)
# 대기는 해당 팀 소속으로 간주합니다.
hr["dep"] = hr["dep"].str.replace("(대기)", "", regex=False)
# "(연구실)", "(착공준비)"는 해당 팀 소속으로 간주합니다.
hr["dep"] = hr["dep"].str.replace("(연구실)", "", regex=False)
hr["dep"] = hr["dep"].str.replace("(착공준비)", "", regex=False)
# 지사 소속 직원은 제외합니다.
hr = hr[hr["type"]!="지사"]
hr.loc[hr["dep"].str.contains("건축PAM"), "dep"] = "건축PAM"
hr.loc[hr["dep"].str.contains("인프라PAM"), "dep"] = "인프라PAM"

email2name = {email:name for email, name in zip(hr["email"], hr["name"])}
email2dep = {email:dep for email, dep in zip(hr["email"], hr["dep"])}
email2hq = {email:hq for email, hq in zip(hr["email"], hr["hq"])}

In [3]:
filenames = glob.glob(f"{pref}전체/*.csv")
dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename))
data = pd.concat(dfs, ignore_index=True)

data = data.drop(["comment_id"], axis=1)
data = data.fillna(0)
data[["post_from", "commenter"]] = data[["post_from", "commenter"]].astype("str")
data[["post_subject_size", "comment_body_size"]] = data[["post_subject_size", "comment_body_size"]].astype("int")
data["post_created_at"] = data["post_created_at"].apply(lambda x:pd.to_datetime(x))
data["comment_created_at"] = data["comment_created_at"].apply(lambda x:pd.to_datetime(x))

## 최근 사용량

In [4]:
# 최근 1개월 간 작성된 업무만 뽑습니다.
recent_posts = data[(data["post_created_at"].dt.year==target_year) & (data["post_created_at"].dt.month==target_month)]
# 타사 직원이 작성한 업무는 제거합니다.
recent_posts = recent_posts[recent_posts["post_from"]!="0"]
# 중복된 업무는 제거합니다.
recent_posts = recent_posts.drop_duplicates(["post_id"])
recent_posts = pd.merge(recent_posts, hr, left_on="post_from", right_on="email", how="outer")
recent_posts = recent_posts[["project_name", "hq", "dep", "name", "post_body_size"]]
recent_post = recent_posts.groupby(["hq", "dep", "name"]).count()["post_body_size"]
recent_post_size = recent_posts.groupby(["hq", "dep", "name"]).sum()["post_body_size"]

# 최근 1개월 간 작성된 댓글만 뽑습니다.
recent_comments = data[(data["comment_created_at"].dt.year==target_year) & (data["comment_created_at"].dt.month==target_month)]
# 타사 직원이 작성한 댓글은 제거합니다.
recent_comments = recent_comments[recent_comments["commenter"]!="0"]
recent_comments = pd.merge(recent_comments, hr, left_on="commenter", right_on="email", how="outer")
recent_comments = recent_comments[["project_name", "hq", "dep", "name", "comment_body_size"]]
recent_comment = recent_comments.groupby(["hq", "dep", "name"]).count()["comment_body_size"]
recent_comment_size = recent_comments.groupby(["hq", "dep", "name"]).sum()["comment_body_size"]

temp = pd.merge(recent_post, recent_comment, left_index=True, right_index=True, how="outer")
temp = pd.merge(temp, recent_post_size, left_index=True, right_index=True)
merge = pd.merge(temp, recent_comment_size, left_index=True, right_index=True)
merge.columns = ["post", "comment", "post_size", "comment_size"]

hr_temp = hr[~hr["hq"].str.contains("회장직속|부회장직속")]
idx = pd.MultiIndex.from_tuples(list(hr_temp.groupby(["hq", "dep", "name"]).groups))
recent = pd.DataFrame(data=0, index=idx, columns=["post", "comment", "post_size", "comment_size"])

for _, row in merge.iterrows():
    recent.loc[row.name] = row
recent = recent.reset_index()
recent = recent.rename({"level_0":"hq", "level_1":"dep", "level_2":"name"}, axis=1)
recent = recent[~recent["dep"].str.contains("회장")]
recent = recent[~recent["dep"].str.contains("사장")]
recent = recent[~recent["dep"].str.contains("본부장")]
recent = recent[~recent["dep"].str.contains("상임자문")]
recent = recent[~recent["dep"].str.contains("비상근")]
# recent = recent.set_index(["hq", "dep", "name"])
# recent.index.names = (None, None, None)

In [5]:
# recent.to_csv(f"{pref}최근 부서별 사용량.csv", encoding="euc-kr", index=False)
# recent.head()

In [6]:
prj2head = {"회장-보고":hr.loc[hr["dep"]=="회장", "email"].item(),
           "부회장-보고":hr.loc[hr["dep"]=="부회장", "email"].item(),
           "대표이사사장-보고":hr.loc[hr["dep"]=="대표이사사장", "email"].item(),
           "개발영업본부-보고":hr.loc[hr["dep"]=="개발영업본부", "email"].item(),
           "건설본부-보고":hr.loc[hr["dep"]=="건설본부", "email"].item(),
           "경영본부-보고":hr.loc[hr["dep"]=="경영본부", "email"].item(),
           "미래혁신본부-보고":hr.loc[hr["dep"]=="미래혁신본부", "email"].item()}
head2prj = {value:key for key, value in prj2head.items()}

# 최근 1개월 간 작성된 댓글만 뽑습니다.
escal = data[(data["comment_created_at"].dt.year==target_year) & (data["comment_created_at"].dt.month==target_month)]
# 위 프로젝트에서 작성된 업무 및 댓글만 뽑습니다.
escal = escal[escal["project_name"].isin(prj2head.keys())]
# 업무별로 댓글을 남긴 사람을 모두 리스트로 만듭니다.
escal["commenters"] = escal.apply(lambda x:escal[escal["post_id"]==x["post_id"]]["commenter"].tolist(), axis=1)
# 보고 아이템 기준으로 중복된 보고를 제거합니다.
escal = escal.drop_duplicates(["post_id"])
# 각 프로젝트마다의 보고 받는 이를  "head" column으로 설정합니다.
escal["head"] = escal["project_name"].map(prj2head)
# 댓글을 남긴 사람들의 리스트에 보고 받는 이를 포함시킵니다.
escal.apply(lambda x:x["commenters"].append(x["head"]), axis=1)
# 댓글을 남긴 사람들 중 본부장 이상만 남깁니다.
escal["commenters"] = escal["commenters"].apply(lambda x:list(set(x) & set(prj2head.values())))
# 본부장 이상에 대해 각각 담당하는 보고방에 대응시킵니다.
escal["heads"] = escal["commenters"].apply(lambda x:[head2prj[el] for el in x])
escal = escal[["post_from", "heads"]]
escal = pd.merge(escal, hr[["email", "hq", "dep", "name"]], left_on="post_from", right_on="email")
escal = escal[["hq", "dep", "name", "heads"]]

df = pd.DataFrame()
for _, row in escal.iterrows():
    for el in row["heads"]:
#         if el not np.nan:
        df = df.append({"hq":row["hq"], "dep":row["dep"], "name":row["name"], "project_name":el}, ignore_index=True)
#         else:
#             pass
escalation = df.groupby(["project_name", "hq", "dep", "name"], as_index=False).size()

# 보고 수가 0인 프로젝트를 데이터에 추가합니다.
missing_set = set(prj2head.keys()) - set(escalation["project_name"])
for missing in missing_set:
    escalation = escalation.append({"project_name":missing, "hq":"-", "dep":"-", "name":"-", "size":0}, ignore_index=True)
    
escalation["project_name"] = escalation["project_name"].str.replace("회장-보고", "회장", regex=False)
escalation["project_name"] = escalation["project_name"].str.replace("부회장-보고", "부회장", regex=False)
escalation["project_name"] = escalation["project_name"].str.replace("대표이사사장-보고", "사장", regex=False)
escalation["project_name"] = escalation["project_name"].str.replace("개발영업본부-보고", "개발영업본부", regex=False)
escalation["project_name"] = escalation["project_name"].str.replace("건설본부-보고", "건설본부", regex=False)
escalation["project_name"] = escalation["project_name"].str.replace("경영본부-보고", "경영본부", regex=False)
escalation["project_name"] = escalation["project_name"].str.replace("미래혁신본부-보고", "미래혁신본부", regex=False)

In [7]:
# escalation.to_csv(f"{pref}보고방별 보고 건수.csv", encoding="euc-kr")
# escalation.head()

## 월별 사용량

In [8]:
data_posts = data.drop_duplicates(["post_id"])
post_month = data_posts.groupby(pd.Grouper(key="post_created_at", freq="M")).size()
post_month.name = "post"
post_size_month = data_posts.groupby(pd.Grouper(key="post_created_at", freq="M")).sum()["post_body_size"]
post_size_month.name = "post_size"

comment_month = data.groupby(pd.Grouper(key="comment_created_at", freq="M")).size()
comment_month.name = "comment"
comment_size_month = data.groupby(pd.Grouper(key="post_created_at", freq="M")).sum()["comment_body_size"]
comment_size_month.name = "comment_size"

temp = pd.merge(post_month, comment_month, left_index=True, right_index=True, how="outer")
temp = pd.merge(temp, post_size_month, left_index=True, right_index=True, how="inner")
month = pd.merge(temp, comment_size_month, left_index=True, right_index=True, how="inner")

In [9]:
# month.to_csv(f"{pref}연월별 사용량.csv", encoding="euc-kr")
# month.tail()

In [10]:
bogo = data[data["project_name"].isin(prj2head.keys())]
bogo = bogo.drop_duplicates(["post_id"])
bogo_month = bogo.groupby(pd.Grouper(key="post_created_at", freq="M")).size()

In [11]:
# bogo_month.to_csv(f"{pref}연월별 보고 건수.csv", encoding="euc-kr")
# bogo_month.tail()

# H-PIC

In [12]:
# filenames = glob.glob("./FINAL/전체/*.csv")
# dfs = []
# for filename in filenames:
#     dfs.append(pd.read_csv(filename))
# hpic = pd.concat(dfs, ignore_index=True)

# hpic["post_id"] = hpic["post_id"].astype("str")
# hpic["post_id"] = hpic["post_id"].str[-10:]

# hpic = hpic.drop(["project_id", "post_subject_size", "comment_id"], axis=1)

# hpic["post_created_at"] = hpic["post_created_at"].apply(lambda x:pd.to_datetime(x))
# hpic["comment_created_at"] = hpic["comment_created_at"].apply(lambda x:pd.to_datetime(x))
# # 가장 바깥의 따옴표를 제거합니다.
# hpic["post_to"] = hpic["post_to"].map(eval)
# hpic["post_cc"] = hpic["post_cc"].map(eval)
# hpic["to"] = hpic["post_to"] + hpic["post_cc"]

# hpic = hpic[hpic["to"].apply(len)!=0]
# hpic = hpic.drop(["post_to", "post_cc"], axis=1)

# hpic.apply(lambda x:x["to"].remove(x["post_from"]) if x["post_from"] in x["to"] else x["to"], axis=1)
# hpic = hpic[hpic["to"].apply(len)!=0]
# hpic["to"] = hpic["to"].apply(lambda x:x[0])

# hpic = hpic.rename({"post_from":"from"}, axis=1)

# hpic["from_hq"] = hpic["from"].map(email2hq)
# hpic["from_dep"] = hpic["from"].map(email2dep)
# hpic["from_name"] = hpic["from"].map(email2name)

# hpic["to_hq"] = hpic["to"].map(email2hq)
# hpic["to_dep"] = hpic["to"].map(email2dep)
# hpic["to_name"] = hpic["to"].map(email2name)

# hpic["commenter"] = hpic["commenter"].map(email2name)

# hpic["from_comment"] = hpic.apply(lambda x:x["comment_body_size"] if x["commenter"]==x["from_name"] else 0, axis=1)
# hpic["to_comment"] = hpic.apply(lambda x:x["comment_body_size"] if x["commenter"]==x["to_name"] else 0, axis=1)

# for post in set(hpic["post_id"]):
#     hpic[hpic["post_id"]==post]

# hpic["from_comment"] = hpic.apply(lambda x:x["comment_body_size"] if x["commenter"]==x["from_name"] else 0, axis=1)
# hpic["to_comment"] = hpic.apply(lambda x:x["comment_body_size"] if x["commenter"]==x["to_name"] else 0, axis=1)

# hpic = hpic[["from_hq", "from_dep", "from_name", "from_comment", "to_hq", "to_dep", "to_name", "to_comment", "comment_created_at"]]

In [13]:
# hpic.to_csv("H-PIC 멘토-멘티 댓글 작성 날짜 및 용량_-210228.csv", encoding="euc-kr")
# hpic.head()